# Assessment 2 MATH70094 Programming for Data Science Autumn 2025

# NAME: GRAHAM DIB and CID: 01709270 

# Question 2 (16 marks)

In this question you continue the analysis of the flight data using Python. 
 
Some helpful functions for this question are `merge`, `apply`, `groupby`, `dropna`.

## Code clarity (1 mark)

There is a famous saying among software developers that code is read more often than it is written. Marks will be awarded (or not awarded) based on the clarity of the code and appropriate use of comments.

## Part A

### Part A(i) (1 mark)

Load the contents of the csv files `flights.csv` and `planes.csv` as dataframes named `flights` and `planes`. Print the first 6 rows from each dataframe. How many flights are there in all?

In [61]:
import pandas as pd
import numpy as np

flights = pd.read_csv("flights.csv")
planes = pd.read_csv("planes.csv")

print(flights.head(6))
print(planes.head(6))

print("total number of flights:", len(flights))

   year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0  2013      1    1     517.0             515        2.0     830.0   
1  2013      1    1     533.0             529        4.0     850.0   
2  2013      1    1     542.0             540        2.0     923.0   
3  2013      1    1     544.0             545       -1.0    1004.0   
4  2013      1    1     554.0             600       -6.0     812.0   
5  2013      1    1     554.0             558       -4.0     740.0   

   sched_arr_time  arr_delay carrier  flight tailnum origin dest  air_time  \
0             819       11.0      UA    1545  N14228    EWR  IAH     227.0   
1             830       20.0      UA    1714  N24211    LGA  IAH     227.0   
2             850       33.0      AA    1141  N619AA    JFK  MIA     160.0   
3            1022      -18.0      B6     725  N804JB    JFK  BQN     183.0   
4             837      -25.0      DL     461  N668DN    LGA  ATL     116.0   
5             728       12.0      UA    1

### Part A(ii) (1 mark)

Find the number of missing entries for each column in `flights`.

In [62]:
missing_entries = flights.isna().sum()
print("missing entries per column:\n", missing_entries)

missing entries per column:
 year                 0
month                0
day                  0
dep_time          8255
sched_dep_time       0
dep_delay         8255
arr_time          8713
sched_arr_time       0
arr_delay         9430
carrier              0
flight               0
tailnum           2512
origin               0
dest                 0
air_time          9430
distance             0
hour                 0
minute               0
time_hour            0
dtype: int64


### Part A(iii) (2 marks)

Add a new column named `long_delayed_AA` to `flights`, which contains for a flight the value `True` if it is a flight by the carrier 'AA', is long haul (`distance` more than 1000 miles) and arrived with more than 30 minutes delay. Otherwise, it contains the value `False`. To check the results after adding the column, print the first 10 rows of `flights` including only the columns `carrier`, `distance`, `arr_delay` and `long_delayed_AA`. For how many flights is `long_delayed_AA` equal to `True`?  

In [63]:
flights["long_delayed_AA"] = (
    (flights["carrier"] == "AA") & (flights["distance"] > 1000) & (flights["arr_delay"] > 30)
)

print(flights[["carrier", "distance", "arr_delay", "long_delayed_AA"]].head(10))

  carrier  distance  arr_delay  long_delayed_AA
0      UA      1400       11.0            False
1      UA      1416       20.0            False
2      AA      1089       33.0             True
3      B6      1576      -18.0            False
4      DL       762      -25.0            False
5      UA       719       12.0            False
6      B6      1065       19.0            False
7      EV       229      -14.0            False
8      B6       944       -8.0            False
9      AA       733        8.0            False


## Part B

There are many missing entries in the `arr_delay` column in `flights`. Your task is to impute as many missing values using information from other variables.

### Part B(i) (3 marks)

To identify which covariates affect `arr_delay`, create three dataframes from `flights` as follows:
- a dataframe named `monthly_stats` with two columns, which in each row contains for one of the 12 months the average arrival delay in that month as well as the total number of flights,
- a dataframe named `hourly_stats` with two columns, which in each row contains for one of the 24 hours in a day the average arrival delay in that month as well as the total number of flights,
- a dataframe named `carrier_stats` with two columns, which in each row contains for one of the carriers the average arrival delay in that month as well as the total number of flights. 

Print all three newly created dataframes. Describe what you notice for each dataframe using one sentence.

In [64]:
monthly_stats = flights.groupby("month", dropna=False)["arr_delay"].agg(['mean', 'size']).rename(columns={'mean': 'mean_arr_delay', 'size': 'total_flights'})
hourly_stats = flights.groupby("hour", dropna=False)["arr_delay"].agg(['mean', 'size']).rename(columns={'mean': 'mean_arr_delay', 'size': 'total_flights'})
carrier_stats = flights.groupby("carrier", dropna=False)["arr_delay"].agg(['mean', 'size']).rename(columns={'mean': 'mean_arr_delay', 'size': 'total_flights'})

print(monthly_stats)
print(hourly_stats)
print(carrier_stats)

print("\n\nFor monthly stats, average arrival delays tend to be higher in holiday months, such as June, July, and December, and the total number of flights is relatively consistent throughout the months.")
print("\n\nFor hourly stats, total flights are highest during the morning and afternoon rush hours, while the average arrival delay accumulates throughout the day, becoming largest in the evening.")
print("\n\nFor carrier stats, average arrival delays vary significantly between different airlines, with some experiencing much higher delays on average, and seemingly no correlation between the mean arrival delay and total number of flights.")

       mean_arr_delay  total_flights
month                               
1            6.129972          27004
2            5.613019          24951
3            5.807577          28834
4           11.176063          28330
5            3.521509          28796
6           16.481330          28243
7           16.711307          29425
8            6.040652          29327
9           -4.018364          27574
10          -0.167063          28889
11           0.461347          27268
12          14.870355          28135
      mean_arr_delay  total_flights
hour                               
1                NaN              1
5          -4.796907           1953
6          -3.384485          25951
7          -5.304472          22821
8          -1.113227          27242
9          -1.451407          20312
10          0.953940          16708
11          1.481930          16033
12          3.489010          18181
13          6.544740          19956
14          9.197650          21706
15         12.

### Part B(iii) (2 marks)

Create a new dataframe `flights_imp` from `flights`. In the new dataframe, impute the missing `arr_delay` entries. To do so, fill in for a given row with a missing `arr_delay` entry the arrival delay averaged over all the flights with the same carrier within the same month and the same hour. After imputation, how many missing entries are left in the column `arr_delay`?

In [65]:
flights_imp = flights.copy()

flights_imp['arr_delay'] = flights_imp['arr_delay'].fillna(
    flights_imp.groupby(['carrier', 'month', 'hour'])['arr_delay'].transform('mean')
)

num_missing_left = flights_imp['arr_delay'].isna().sum()

print("After imputation, number of missing entries that are left in the column 'arr_delay' is:", num_missing_left)



After imputation, number of missing entries that are left in the column 'arr_delay' is: 1


### Part B(iii) (1 mark)

Using the `describe` method from Pandas, discuss if the distribution (quantiles, min, max) of the values in `arr_delay` has changed after imputation.

In [66]:
before_imputation_mean = flights['arr_delay'].describe()
after_imputation_mean = flights_imp['arr_delay'].describe()

print("Before imputation 'arr_delay' stats:\n", before_imputation_mean)
print("After imputation 'arr_delay' stats:\n", after_imputation_mean)

print("\n\n The imputation resulted in a slightly shifted distribution. the min and max values remain unchanged, but the mean, median, and quartiles have all increased slightly, resulting in the central tendency of the distribution being higher after imputation. ")

Before imputation 'arr_delay' stats:
 count    327346.000000
mean          6.895377
std          44.633292
min         -86.000000
25%         -17.000000
50%          -5.000000
75%          14.000000
max        1272.000000
Name: arr_delay, dtype: float64
After imputation 'arr_delay' stats:
 count    336775.000000
mean          7.115579
std          44.104194
min         -86.000000
25%         -16.000000
50%          -4.000000
75%          15.000000
max        1272.000000
Name: arr_delay, dtype: float64


 The imputation resulted in a slightly shifted distribution. the min and max values remain unchanged, but the mean, median, and quartiles have all increased slightly, resulting in the central tendency of the distribution being higher after imputation. 


## Part C

You are asked to study the size of planes appearing in `flights`. 

### Part C(i) (1 mark)

Add a new column named `size_category` to `planes` using the `apply` function from Pandas. The new column should indicate if a plane is `Small` (less than 100 seats), `Medium` (between 100 and 180 seats), 'Large' (between 181 and 250 seats) or 'Jumbo' (more than 250 seats). Keep the `%%timeit` at the beginning of the code cell below to measure the speed of executing your code.

In [71]:
%%timeit 

def calc_plane_size(row):
    if row['seats'] < 100:
        return 'Small'
    elif 100 <= row['seats'] <= 180:
        return 'Medium'
    elif 181 <= row['seats'] <= 250:
        return 'Large'
    elif row['seats'] > 250:
        return 'Jumbo'
    
    
planes['size_category'] = planes.apply(calc_plane_size, axis=1)

10.2 ms ± 49.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Part C(ii) (2 mark)

Repeat the task from C(i) but using only Numpy operations, to add a new column `size_category_np` to `planes`. Again, use `%%timeit` when running your code. What do you notice in comparison to the result in C(i) in terms of the running time?

In [ ]:
%%timeit

planes['size_category_np'] = np.select(
    [planes['seats'] < 100,
     (planes['seats'] >= 100) & (planes['seats'] <= 180),
     (planes['seats'] >= 181) & (planes['seats'] <= 250),
     planes['seats'] > 250],
    ['Small', 'Medium', 'Large', 'Jumbo'],
    default='Unknown'
)   

# The np.select method is significantly faster and has a much smaller std.dev than the apply method with a custom function.
# np.select does vectorised operations in compiled code, while apply is python level operations looping over rows.

370 µs ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Part C(iii) (1 mark)

Using the `merge` method from Pandas, add the `size_category` column from `planes` to `flights`. 

In [76]:
flights = flights.merge(
    planes[['tailnum', 'size_category']],
    on='tailnum',
    how='left'
)

flights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,long_delayed_AA,size_category
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,False,Medium
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,False,Medium
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,True,Medium
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,False,Large
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,False,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,...,NaN,JFK,DCA,NaN,213,14,55,2013-09-30 14:00:00,False,NaN
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,...,NaN,LGA,SYR,NaN,198,22,0,2013-09-30 22:00:00,False,NaN
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,...,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30 12:00:00,False,NaN
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,...,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30 11:00:00,False,NaN


### Part C(iv) (1 marks)

Determine for each size category the destination (`dest`) with the most flights. Which carrier uses the most `Large` planes?

In [101]:
flights_grouped_size = flights.groupby(['size_category', 'dest']).size()
top_dest = flights_grouped_size.groupby(level=0).idxmax().reset_index(name='most_frequent_dest')
top_dest['most_frequent_dest'] = top_dest['most_frequent_dest'].apply(lambda t: t[1])

flights_grouped_carrier = flights.groupby(['size_category', 'carrier']).size()
flights_grouped_carrier_large = flights_grouped_carrier.loc['Large']

carrier_with_most_large_flights = flights_grouped_carrier_large.idxmax()
carrier_with_most_large_flights_count = flights_grouped_carrier_large.max()

print("Most frequent destination for each plane size category:\n", top_dest)

print("\n\nThe carrier that uses the most large planes is:", carrier_with_most_large_flights)
print("The number of flights taken on large planes by this carrier is:", carrier_with_most_large_flights_count)


Most frequent destination for each plane size category:
   size_category most_frequent_dest
0         Jumbo                LAX
1         Large                FLL
2        Medium                ATL
3         Small                BOS


The carrier that uses the most large planes is: B6
The number of flights taken on large planes by this carrier is: 34063
